In [1]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import lightgbm as lgb
from xgboost.sklearn import XGBClassifier
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import os
from tqdm import tqdm
import xgboost as xgb
import catboost
import torch
import torch.nn as nn
import torch.nn.functional as F

class mlp(nn.Module):
    def __init__(self, input_size, output_size):
        super(mlp, self).__init__()
        self.fc1 = nn.Linear(input_size, 40)
        self.fc2 = nn.Linear(40, 4)
    def forward(self, x):
        x = self.fc1(x)
        x = F.softmax(x)
        x = self.fc2(x)
        x = F.softmax(x)
        return x

def get_auc(y_test, y_pred):
    cnt = 0
    for i in range(y_test.shape[0]):
        if y_test[i] == y_pred[i]:
            cnt += 1
    return float(cnt) / y_test.shape[0]

def get_auc_others(y_test, y_pred):
    cnt = 0
    cnt_sum = 0
    for i in range(y_test.shape[0]):
        if y_test[i] == y_pred[i] and y_test[i] != 3:
            cnt += 1
        if y_test[i] != 3:
            cnt_sum += 1
    return float(cnt) / cnt_sum

seed = 2021

def cv_model(train_x, train_y, test_x, clf_name):
    folds = 5
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros((train_x.shape[0], 4))
    test = np.zeros((test_x.shape[0], 4))

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y.iloc[train_index], train_x.iloc[valid_index], train_y.iloc[valid_index]

        if clf_name == "lgb":
            train_matrix = lgb.Dataset(trn_x, label=trn_y)
            valid_matrix = lgb.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'multiclass',
                'num_class': 4,
                'metric': 'multi_error',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.1,
                'seed': seed,
                'nthread': 28,
                'n_jobs':24,
                'silent': True,
                'verbose': -1,
            }

            model = lgb.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], verbose_eval=200,early_stopping_rounds=200)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
                
        if clf_name == "xgb":
            model = XGBClassifier(
                    learning_rate=0.1,
                   n_estimators=1000,         # 树的个数--1000棵树建立xgboost
                    max_depth=6,               # 树的深度
                   min_child_weight = 1,      # 叶子节点最小权重
                   gamma=0.,                  # 惩罚项中叶子结点个数前的参数
                   subsample=0.8,             # 随机选择80%样本建立决策树
                   colsample_btree=0.8,       # 随机选择80%特征建立决策树
                   objective='multi:softprob', # 指定损失函数
                   scale_pos_weight=1,        # 解决样本个数不平衡的问题
                   random_state=seed    
            )
            model.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], eval_metric='mlogloss', verbose=True, early_stopping_rounds=100)
            val_pred = model.predict_proba(val_x)
            test_pred = model.predict_proba(test_x)
                 
        if clf_name == "cat":
            params = {'learning_rate': 0.05, 'depth': 5, 'l2_leaf_reg': 10, 'bootstrap_type': 'Bernoulli','loss_function':'MultiClass',
                      'od_type': 'Iter', 'od_wait': 50, 'random_seed': 11, 'allow_writing_files': False}
            model = catboost.CatBoostClassifier(iterations=20000, **params)
            model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                      cat_features=[], use_best_model=True, verbose=500)
            
            val_pred  = model.predict_proba(val_x)
            test_pred = model.predict_proba(test_x)
        
        if clf_name == "mlp":
            model = mlp(trn_x.shape[1], 4)
            criterion = nn.CrossEntropyLoss()
            optimizer = torch.optim.SGD(params = model.parameters(),lr = 0.01)
            target = torch.from_numpy(trn_y.values).long()
            for epoch in range(1000):
                optimizer.zero_grad()
                output = model(torch.from_numpy(trn_x.values).to(torch.float32))
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
            val_pred = model.forward(torch.from_numpy(val_x.values).to(torch.float32)).detach().numpy()
            test_pred = model.forward(torch.from_numpy(test_x.values).to(torch.float32)).detach().numpy()
            
        train[valid_index] = val_pred
        test = test_pred / kf.n_splits
    return train, test


In [2]:
data = pd.read_csv('data_train_knn_all_append.csv', index_col=0)
data.drop(columns=['PARFORM'], inplace=True)
#data = data.astype(float)
x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=['entid', 'CaseType'], axis=1),data.CaseType, test_size=0.2)
train = np.zeros((x_train.shape[0], 4))
test = np.zeros((x_test.shape[0], 4))
model_set = ['mlp', 'mlp', 'lgb', 'xgb', 'cat']
for model in model_set:
    train_temp, test_temp = cv_model(x_train, y_train, x_test, model)
    train += train_temp
    test += test_temp
train_data_final_xgb = xgb.DMatrix(train, y_train)
test_data = xgb.DMatrix(test)
params = {'booster': 'gbtree',
          'objective': 'multi:softmax',
          'num_class': 4,
            'eval_metric': 'mlogloss',
          'gamma': 1,
          'min_child_weight': 1.5,
          'max_depth': 5,
          'lambda': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'colsample_bylevel': 0.7,
          'eta': 0.04,
          'tree_method': 'exact',
          'seed': seed,
          'nthread': 36,
}
final_model = xgb.train(params, train_data_final_xgb, num_boost_round=1000)
y_pred = final_model.predict(test_data, ntree_limit=final_model.best_iteration)
print(y_pred.shape)
print(y_test.shape)
print(get_auc(y_test.values, y_pred))

************************************ 1 ************************************


<ipython-input-1-8a59f5283a98>:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)
<ipython-input-1-8a59f5283a98>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


************************************ 2 ************************************
************************************ 3 ************************************
************************************ 4 ************************************
************************************ 5 ************************************
************************************ 1 ************************************
************************************ 2 ************************************
************************************ 3 ************************************
************************************ 4 ************************************
************************************ 5 ************************************
************************************ 1 ************************************
[LightGBM] [Warning] num_threads is set with n_jobs=24, nthread=28 will be ignored. Current value: num_threads=24
[LightGBM] [Warning] Unknown parameter: silent


/Users/fanhang/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1222: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  _log_warning('{0} keyword has been found in `params` and will be ignored.\n'


Training until validation scores don't improve for 200 rounds
[200]	training's multi_error: 0.00824588	valid_1's multi_error: 0.0718563
[400]	training's multi_error: 0.00224888	valid_1's multi_error: 0.0688623
Early stopping, best iteration is:
[256]	training's multi_error: 0.00449775	valid_1's multi_error: 0.0688623
************************************ 2 ************************************
[LightGBM] [Warning] num_threads is set with n_jobs=24, nthread=28 will be ignored. Current value: num_threads=24
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 200 rounds
[200]	training's multi_error: 0.0104948	valid_1's multi_error: 0.0748503
Early stopping, best iteration is:
[87]	training's multi_error: 0.0427286	valid_1's multi_error: 0.0748503
************************************ 3 ************************************
[LightGBM] [Warning] num_threads is set with n_jobs=24, nthread=28 will be ignored. Current value: num_threads=24
[LightGBM] [

...

[7]	validation_0-mlogloss:0.67699
[8]	validation_0-mlogloss:0.63162
[9]	validation_0-mlogloss:0.59447
[10]	validation_0-mlogloss:0.55933
[11]	validation_0-mlogloss:0.52937
[12]	validation_0-mlogloss:0.50101
[13]	validation_0-mlogloss:0.47596
[14]	validation_0-mlogloss:0.45378
[15]	validation_0-mlogloss:0.43473
[16]	validation_0-mlogloss:0.41634
[17]	validation_0-mlogloss:0.40079
[18]	validation_0-mlogloss:0.38714
[19]	validation_0-mlogloss:0.37562
[20]	validation_0-mlogloss:0.36249
[21]	validation_0-mlogloss:0.35167
[22]	validation_0-mlogloss:0.34233
[23]	validation_0-mlogloss:0.33326
[24]	validation_0-mlogloss:0.32558
[25]	validation_0-mlogloss:0.31892
[26]	validation_0-mlogloss:0.31299
[27]	validation_0-mlogloss:0.30697
[28]	validation_0-mlogloss:0.30053
[29]	validation_0-mlogloss:0.29511
[30]	validation_0-mlogloss:0.29178
[31]	validation_0-mlogloss:0.28664
[32]	validation_0-mlogloss:0.28335
[33]	validation_0-mlogloss:0.27856
[34]	validation_0-mlogloss:0.27585
[35]	validation_0-mlogl

[53]	validation_0-mlogloss:0.27855
[54]	validation_0-mlogloss:0.27933
[55]	validation_0-mlogloss:0.27820
[56]	validation_0-mlogloss:0.27873
[57]	validation_0-mlogloss:0.27761
[58]	validation_0-mlogloss:0.27697
[59]	validation_0-mlogloss:0.27814
[60]	validation_0-mlogloss:0.27927
[61]	validation_0-mlogloss:0.27880
[62]	validation_0-mlogloss:0.27884
[63]	validation_0-mlogloss:0.27991
[64]	validation_0-mlogloss:0.27996
[65]	validation_0-mlogloss:0.27972
[66]	validation_0-mlogloss:0.27998
[67]	validation_0-mlogloss:0.28002
[68]	validation_0-mlogloss:0.28031
[69]	validation_0-mlogloss:0.28012
[70]	validation_0-mlogloss:0.28127
[71]	validation_0-mlogloss:0.28171
[72]	validation_0-mlogloss:0.28280
[73]	validation_0-mlogloss:0.28436
[74]	validation_0-mlogloss:0.28463
[75]	validation_0-mlogloss:0.28447
[76]	validation_0-mlogloss:0.28538
[77]	validation_0-mlogloss:0.28662
[78]	validation_0-mlogloss:0.28720
[79]	validation_0-mlogloss:0.28648
[80]	validation_0-mlogloss:0.28686
[81]	validation_0-ml

[128]	validation_0-mlogloss:0.31431
[129]	validation_0-mlogloss:0.31506
[130]	validation_0-mlogloss:0.31629
[131]	validation_0-mlogloss:0.31716
[132]	validation_0-mlogloss:0.31773
[133]	validation_0-mlogloss:0.31852
[134]	validation_0-mlogloss:0.31822
[135]	validation_0-mlogloss:0.31893
[136]	validation_0-mlogloss:0.31929
[137]	validation_0-mlogloss:0.31961
[138]	validation_0-mlogloss:0.31984
[139]	validation_0-mlogloss:0.31990
[140]	validation_0-mlogloss:0.32035
[141]	validation_0-mlogloss:0.32055
[142]	validation_0-mlogloss:0.32139
[143]	validation_0-mlogloss:0.32176
[144]	validation_0-mlogloss:0.32229
[145]	validation_0-mlogloss:0.32282
[146]	validation_0-mlogloss:0.32324
[147]	validation_0-mlogloss:0.32360
[148]	validation_0-mlogloss:0.32379
[149]	validation_0-mlogloss:0.32451
[150]	validation_0-mlogloss:0.32515
************************************ 4 ************************************
[11:22:57] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "colsamp

[38]	validation_0-mlogloss:0.31194
[39]	validation_0-mlogloss:0.31038
[40]	validation_0-mlogloss:0.31100
[41]	validation_0-mlogloss:0.31105
[42]	validation_0-mlogloss:0.31145
[43]	validation_0-mlogloss:0.31243
[44]	validation_0-mlogloss:0.31277
[45]	validation_0-mlogloss:0.31251
[46]	validation_0-mlogloss:0.31427
[47]	validation_0-mlogloss:0.31521
[48]	validation_0-mlogloss:0.31575
[49]	validation_0-mlogloss:0.31484
[50]	validation_0-mlogloss:0.31573
[51]	validation_0-mlogloss:0.31685
[52]	validation_0-mlogloss:0.31746
[53]	validation_0-mlogloss:0.31876
[54]	validation_0-mlogloss:0.31959
[55]	validation_0-mlogloss:0.32103
[56]	validation_0-mlogloss:0.32223
[57]	validation_0-mlogloss:0.32368
[58]	validation_0-mlogloss:0.32349
[59]	validation_0-mlogloss:0.32476
[60]	validation_0-mlogloss:0.32571
[61]	validation_0-mlogloss:0.32721
[62]	validation_0-mlogloss:0.32849
[63]	validation_0-mlogloss:0.32911
[64]	validation_0-mlogloss:0.33044
[65]	validation_0-mlogloss:0.33071
[66]	validation_0-ml

/Users/fanhang/opt/anaconda3/lib/python3.8/site-packages/xgboost/core.py:101: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test.values, y_pred))

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
G=nx.Graph()
point=[0,1,2,3,4,5,6]
G.add_nodes_from(point)
edglist=[]
N = [[0, 3, 5, 1],[1, 5, 4, 3],[2, 1, 3, 5],[3, 5, 1, 4],[4, 5, 1, 3],[5, 3, 4, 1],[6, 3, 1, 4]]
for i in range(7):
  for j in range(1,4):
    edglist.append((N[i][0],N[i][j]))
G=nx.Graph(edglist)
position = nx.circular_layout(G)
nx.draw_networkx_nodes(G,position, nodelist=point, node_color="r")
nx.draw_networkx_edges(G,position)
nx.draw_networkx_labels(G,position)
plt.show()